## Cell 1 — 配置（范围、输出目录）

In [1]:
# === Config (single key) ===

API_KEY = "sk-AAWv68b047ef83b2b12091"   # ←← 替换成你的真实 key

# 注意：disease_id 范围通常是 1–100（免费档）
START_ID = 1
END_ID   = 100

# 限速与重试
SLEEP_BETWEEN = 1
MAX_RETRIES   = 5
BACKOFF_BASE  = 1.6

# 输出目录 & 文件名模式
from pathlib import Path
OUT_DIR = Path("01_raw_data/04_plant_diseases")
FILENAME_PATTERN = "plant_disease_{disease_id}.json"
OUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"Key: ****{API_KEY[-6:]}, 疾病ID范围: {START_ID}～{END_ID}, 输出目录: {OUT_DIR.resolve()}")

Key: ****b12091, 疾病ID范围: 1～100, 输出目录: E:\05_YZH_DS\02_Monash_DS\2025_S2_FIT5120_Industry_Experience_Studio_Project\06_main_project\03_github_submission\03_github_submission\2025-08-SDG13-Plant-X-Website\01_data_wrangling\01_raw_data\04_plant_diseases


## Cell 2 — 工具函数

In [2]:
import time
import json
import requests
from typing import Optional, Dict, Any

def save_json(path: Path, data: Dict[str, Any]):
    path.parent.mkdir(parents=True, exist_ok=True)
    path.write_text(json.dumps(data, ensure_ascii=False, indent=2), encoding="utf-8")

def build_filepath(disease_id: int) -> Path:
    return OUT_DIR / FILENAME_PATTERN.format(disease_id=disease_id)

## Cell 3 — 单条抓取函数（含重试、指数退避、429/5xx 处理）

In [3]:
def fetch_plant_disease(disease_id: int, api_key: str) -> Optional[dict]:
    """
    成功 -> 返回 dict
    404   -> 返回 {"__missing__": True, "id": disease_id}
    None  -> 触发主循环早停（例如配额/IP 限制，多次429，或 x-ratelimit-remaining=0）
    """
    base_url = "https://perenual.com/api/pest-disease-list"
    attempt = 0
    consecutive_429 = 0

    while attempt <= MAX_RETRIES:
        try:
            resp = requests.get(
                base_url,
                params={"id": disease_id, "key": api_key},
                headers={"accept": "application/json"},
                timeout=30
            )

            remaining = resp.headers.get("x-ratelimit-remaining")
            if remaining is not None:
                try:
                    if int(remaining) <= 0:
                        print(f"[ID {disease_id}] x-ratelimit-remaining=0 → 安全早停。")
                        return None
                except ValueError:
                    pass

            if resp.status_code == 200:
                return resp.json()

            if resp.status_code == 404:
                print(f"[ID {disease_id}] 404：该 disease_id 不存在或无数据。")
                return {"__missing__": True, "id": disease_id}

            if resp.status_code == 429:
                consecutive_429 += 1
                if consecutive_429 >= 3:
                    print(f"[ID {disease_id}] 连续 429 次，推断配额/IP 限制 → 安全早停。")
                    return None
                wait = (BACKOFF_BASE ** attempt) + 0.2 * attempt
                print(f"[ID {disease_id}] 429 Too Many Requests，等待 {wait:.1f}s 后重试。")
                time.sleep(wait)
                attempt += 1
                continue

            if resp.status_code in (500, 502, 503, 504):
                wait = (BACKOFF_BASE ** attempt) + 0.2 * attempt
                print(f"[ID {disease_id}] {resp.status_code} 服务端错误，等待 {wait:.1f}s 重试。")
                time.sleep(wait)
                attempt += 1
                continue

            print(f"[ID {disease_id}] HTTP {resp.status_code}: {resp.text[:200]}")
            return {"__error_status__": resp.status_code, "id": disease_id}

        except requests.RequestException as e:
            wait = (BACKOFF_BASE ** attempt) + 0.2 * attempt
            print(f"[ID {disease_id}] 网络异常（{type(e).__name__}）：{e}，等待 {wait:.1f}s 重试。")
            time.sleep(wait)
            attempt += 1

    print(f"[ID {disease_id}] 重试耗尽。")
    return {"__error_retries_exhausted__": True, "id": disease_id}

## Cell 4 — 主循环（断点续跑、跳过已有文件、智能停机）

In [4]:
from tqdm.auto import tqdm
import random

downloaded = 0
skipped = 0

for disease_id in tqdm(range(START_ID, END_ID + 1), desc="Downloading Plant Diseases"):
    fp = build_filepath(disease_id)
    if fp.exists():
        skipped += 1
        continue

    data = fetch_plant_disease(disease_id, API_KEY)
    if data is None:
        print("推断配额/限流 → 安全早停。")
        break

    save_json(fp, data)
    downloaded += 1
    time.sleep(SLEEP_BETWEEN + random.uniform(0.0, 0.6))

print(f"新增下载 {downloaded} 个，跳过(已存在) {skipped} 个。输出目录：{OUT_DIR.resolve()}")

c:\Users\zyyin1\anaconda3\envs\python_for_data_analysis\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

新增下载 0 个，跳过(已存在) 100 个。输出目录：E:\05_YZH_DS\02_Monash_DS\2025_S2_FIT5120_Industry_Experience_Studio_Project\06_main_project\03_github_submission\03_github_submission\2025-08-SDG13-Plant-X-Website\01_data_wrangling\01_raw_data\04_plant_diseases


In [5]:
# import shutil
# from google.colab import files

# shutil.make_archive("04_plant_diseases", 'zip', "01_raw_data/04_plant_diseases")
# files.download("04_plant_diseases.zip")